<a href="https://colab.research.google.com/github/dtht2d/bispectrum_component/blob/main/bispectrum/optimization/test_functions_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 30

Autosaving every 30 seconds


# **Code optimization:**  Compile function used in computing bispectrum components with/without Numba

**Numba**
- An open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.
- Designed to be used with NumPy arrays and functions. Numba generates specialized code for different array data types and layouts to optimize performance.


In [2]:
pip install sympy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.7 MB/s eta 0:00:00
time: 528 µs (started: 2023-03-15 04:23:56 +00:00)


In [40]:
!pip install pandas 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 3.14 s (started: 2023-03-15 05:58:40 +00:00)


## **a. Wigner D function**
Ref.[5], chapter 4.3-eq.1

$$
D^{j}_{mm'}(\theta_0,\theta,\phi)=e^{-im\theta_0}d^{j}_{mm'}(\theta)e^{-im'\phi}
$$
Choose 4.3.1 eq(4) to compute $d^{j}_{mm'}(\theta)$

\begin{align*}d^j_{mm'}{(\theta)}= [(j+m)!(j-m)!(j+m')!(j-m')!]^{\frac{1}{2}} \\ \times \displaystyle\sum_k(-1)^k\frac{(cos\frac{\theta}{2})^{2j-2k+m-m'}(sin\frac{\theta}{2})^{2k-m+m'}}{k!(j+m-k)!(j-m'-k)!(m'-m+k))} \end{align*}
**Note:** $k$ runs over all integer values for which factorial arguments are non-negative. The sums contain $(N+1)$  terms where N is the minimum of $j+m, j-m, j+m'$ and $j-m'$. 

**Finding** $k_{max}, k_{min}:$

\begin{align*} j+m-k &\geq 0 &\\ k &\leq j-m \leq j+m \\ j-m'-k &\geq 0 &\\ k &\leq j-m'  \\ k_{min}&= [\text{int}(\ j+m \ ), \text{int}(\ j-m' \ 
)] \\ m-m'+k&\geq 0 \\ k &\geq m'-m &\\ k_{max}&=\text{int}[0,m'-m] \end{align*} 


### **Compute class function without Numba**

In [6]:
import numpy as np
import cmath

def fact(n):
    """
    This function is used to calculate factorial of a number by using
    an iterative approach instead of recursive approach
    """
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result


class Wigner_D:
    """
    Args:
        j (scalar): angular momentum
        m (scalar): eigenvalue of angular momentum
        mp (scalar): eigenvalue of j along rotated axis
        theta_0 (scalar): first angle of rotation [0, pi]
        theta (scalar): second angle of rotation [0, pi]
        phi (scalar): third angle of rotation [0, 2*pi]
    Returns: complex number, Wigner D function
    ==========================Reference==================================
    [5] Chapter 4.3-(p.76,eq.1)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    """
    def __init__(self, j, m, mp, theta_0, theta, phi):
        if j < 0 or not np.isclose(j, int(j)) or (j % 1 == 0.5 and (m % 1 != 0 or mp % 1 != 0)):
            raise ValueError("Invalid input parameters: j must be a non-negative integer or half-integer, "
                             "m and mp must be between -j and j.")
        if theta_0 < 0 or theta_0 > np.pi or theta < 0 or theta > np.pi or phi < 0 or phi > 2 * np.pi:
            raise ValueError(
                "Invalid input parameters: theta_0, theta, and phi must be within [0, pi] and [0, 2pi], respectively.")
        self.j = j
        self.m = m
        self.mp = mp
        self.theta_0 = theta_0
        self.theta = theta
        self.phi = phi
    def compute_dsmall(self):
        """
        This method is used to calculate the Wigner d small- real function involving trigonometric functions
        ==========================Reference==================================
        [5] Chapter 4.3.1-(p.76,eq.4)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Returns: Wigner d - real function
        """
        kmax = max(0, self.m - self.mp)
        kmin = min(self.j + self.m, self.j - self.mp)
        term1 = np.sqrt(fact(self.j + self.m) * fact(self.j - self.m) * fact(self.j + self.mp) * fact(self.j - self.mp))
        sum = 0
        for k in range(kmax, kmin + 1):
            numerator = (-1) ** k * (cmath.cos(self.theta / 2)) ** (2 * self.j - 2 * k + self.m - self.mp) * \
                        (cmath.sin(self.theta / 2)) ** (2 * k - self.m + self.mp)
            denominator = fact(k) * fact(self.j + self.m - k) * fact(self.j - self.mp - k) * fact(self.mp - self.m + k)
            sum += numerator / denominator
        return sum*term1

    def wigner_D(self):
        term1 = np.exp(-1j * self.m * self.theta_0)
        term2 = self.compute_dsmall()
        term3 = np.exp(-1j * self.mp * self.phi)
        result = term1 * term2 * term3
        return result



time: 4.58 ms (started: 2023-03-15 04:23:57 +00:00)


**Example**

In [7]:
import numpy as np
j, m, mp, theta_0, theta,phi= 1, 1, 0, np.pi, np.pi/2, 0
# Calculate the Wigner D function using our function
WD = Wigner_D(j, m, mp, theta_0, theta,phi)
wd = WD.wigner_D()
print(wd)

(0.7071067811865476+8.659560562354934e-17j)
time: 7.62 ms (started: 2023-03-15 04:23:57 +00:00)


In [8]:
from sympy import *
from sympy.physics.quantum.spin import Rotation
#Calculate the Wigner D matrix with Sympy
rot1 = Rotation.D(j, m, mp, theta_0, theta,phi)
wignerD_sympy = rot1.doit()
print(N(wignerD_sympy))

0.707106781186548 + 8.65956056235493e-17*I
time: 1.27 s (started: 2023-03-15 04:23:57 +00:00)


**Comment:** Our function run faster than Sympy function

### **b. Compute Coupling Coefficients $H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}$analogous to Clebsch -Gordan coeffcients**

**Clebsch- Gordan coefficient**

Ref.[3], Eq.(5) and Ref.[1], Eq.(5)

$$
C^{jm}_{{j_1}{m_1}{j_2}{m_2}} C^{jm'}_{{j_1}{m'_1}{j_2}{m'_2}} \equiv H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}} 
$$

**Definition**

Ref.[5] p.235

A Clebsch-Gordan coefficients are vector addition coefficients. They play an important role in decomposition of reducible representations of rotational….Let angular momentum $j_1$ and $j_2$ with projections on $m_1$  and $m_2$ on the quantization axis. represents **the probability amplitude that $j_1$ and $j_2$ are coupled into a resultant angular momentum $j$ with projection $m$.**


$$
(j_1,m_1)\otimes(j_2,m_2) \to (jm) 
$$

$$
C\equiv\langle{j_1m_1j_2m_2\vert jm}⟩
$$

$$
\langle{j_1j_2m_2\vert jm}⟩=(-1)^{j_1-j_2+m}\sqrt{2j+1}\begin{pmatrix}j_1&j_2&j \\ m_1&m_2&{-m}\end{pmatrix} 
$$

The $3jm$ symbols are related to Clebsch-Gordan coefficients. The inverse relation:

Ref[5]. Ch8.1.2, eq.(12)

$$
C^{jm}_{j_1m_1j_2m_2}=(-1)^{j_1-j_2+m}\sqrt{2j+1}\begin{pmatrix}j_1&j_2&j \\ m_1&m_2&{-m}\end{pmatrix} 
$$

Ref[12]. Ch3.12, eq(3.171)

$$
\begin{align*} C_{j_1m_1j_2m_2}^{jm} &= \ \delta(m_1 +m_2,m) \\ & \times \left[\frac{(2j+1)(j+j_1-j_2)!(j-j_1+j_2)!(j_1+j_2-j)!}{(j+j_1+j_2+1)!} \right]^{\frac{1}{2}} \\ & \times \left[ \frac{(j+m)!(j-m)!}{(j_1+m_1)!(j_1-m_1)!(j_2+m_2)!(j_2-m_2)!}\right]^{\frac{1}{2}} \\ & \times \displaystyle\sum_s\frac{(-1)^{j_2+m_2+s}(j_2+j+m_1-s)!(j_1-m_1+s)!}{s!(j-j_1+j_2-s)!(j+m-s)!(j_1-j_2-m+s)!} \end{align*}
$$

**Condition for $j_1,j_2,j,m_1,m_2,m$**

In accordance with vector addition rules $j_1+j_2=j$, unless the triangular conditions (triangular in equalities) are fulfilled

(1)$|j_1-j_2|\ \le j \le j_1+j_2$ 

$j= 0, \frac{1}{2},1,...\infty$  and $m, m' = -j,-j+1, ...,j-1,j$

(2)$m_1+m_2=m \ \text{and} \ j_1 +j_2=j$

(3)$|m_1| \le j_1, \ |m_2| \le j_2,\ |m| \le j$ 

(4) $j_1,j_2,j$ not exceed a positive integer $J \ \text{is}$

(5) $j_1+j_2-j$ not half integer

(6) $m_1,m_2,m$ are integer or half-integer (positive or negative) numbers

(7) $j_1,j_2,j$ are integer or half integer non negative numbers 

(8) $j_1 +m_1, \ j_2 +m_2, \ j +m, \ j_1+j_2+j$  are integer non-negative numbers
**Condition for $s$ :** such that nowhere does a factorial of negative number appear

$$
\begin{align*} j_2+j+m_1-s  &\geq 0 \rightarrow \ s \leq j_1+j+m_1 \\ j_1-m_1+s &\geq 0 \rightarrow \ s \geq m_1-j_1 \\ j-j_1+j_2-s &\geq 0 \rightarrow \ s \leq j-j_1+j_2 \\ j+m-s &\geq 0 \rightarrow \ s \leq j+m \\ j_1-j_2-m+s &\geq 0 \rightarrow \ s \geq -j_1+j_2+m\end{align*}
$$

$s_{min}=max(0,m_1-j_1,-j_1+j_2+m)$

$s_{max} = min(j_1+j+m_1, j-j_1+j_2,j+m)$

$$
s_{range}=[s_{min},s_{max}+1]
$$

**Note:** add `+1` to `smax`since the `range()`function does not include the endpoint, and we want to include it in the loop

**Parameter:**

$j_1, j_2$ : Angular momenta of states 1 and 2

$j,m\ :$  Total angular momentum of $(j_1+j_2)$

$m_1, \ m_2, \ m:$  Eigenvalues w.r.t to anglar momentum $j_1, \ j_2, \ j$

$m_1', \ m_2', \ m':$ Eigenvalues w.r.t $j_1, \ j_2, \ j$ along rotated axis


### **Compute $C_{j_1m_1j_2m_2}^{jm}, \ C_{j_1m_1j_2m_2}^{jm'}$**

In [13]:
import numpy as np
import cmath
def fact(n):
    """
    This function is used to calculate factorial of a number by using
    an iterative approach instead of recursive approach
    """
    return np.prod(np.arange(1, n + 1))
class Clebsch_Gordan:
    """
    Definition:
    A Clebsch-Gordan coefficients are vector addition coefficients. They play an important role in decomposition of
    reducible representations of rotation. Let j1 and j2 with projections on m1 and m2 on the quantization axis.
    The coefficients represent the probability amplitude that j1 and j2 are coupled into a resultant angular momentum
    j with projection m.
    Args:
        j1 (scalar): angular momentum
        j2 (scalar): angular momentum
        j (scalar): angular momentum
        m1 (scalar): eigenvalue of angular momentum
        m2 (scalar): eigenvalue of angular momentum
        m (scalar): eigenvalue of angular momentum
    Returns: complex number, Clebsh Gordan function
    ==========================Reference==================================
    [5] Chapter 8 D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
    [12] Chapter 3 Biedenharn, L., & Louck, J.D. ,
        Encyclopedia of Mathematics and its Applications (1981)
    """
    def __init__(self, j1, j2, j, m1, m2, m):
        self.j1 = j1
        self.j2 = j2
        self.j = j
        self.m1 = m1
        self.m2 = m2
        self.m = m
        self.J = j1 + j2 + j
        #Condition 1 & 2 & 5
        if not (abs(j1 - j2) <= j <= j1 + j2 and m1 + m2 == m and j1 + j2 - j % 1 != 0.5):
            raise ValueError("Invalid input parameters: j1, j2, j, m1, m2, and m must satisfy the triangle inequality.\ "
                             "j1+j2-j must not be a half-integer")
        #Condition 3 & 6
        if not all(abs(x) <= y and (x % 0.5 == 0 or x % 1 == 0) for x, y in zip([m1, m2, m], [j1, j2, j])):
            raise ValueError("Invalid input parameters: |m1| <= j1, |m_2| <= j2, |m| <= j\ "
                             "m1, m2, m must be integer or half-integer (positive or negative) numbers")
        # Condition 4
        J =(j1+j2+j)
        if J < (int(j1+j2+j)) and J <0:
            raise ValueError("Invalid input parameters: j1, j2, j must not exceed a positive integer J")
        # Condition 7
        if not all(
                isinstance(x, (int, float, np.integer, np.floating)) and x >= 0 and (x % 0.5 == 0 or x % 1 == 0) for x
                in [j1, j2, j]):
            raise ValueError(
                "Invalid input parameters: j1, j2, j must be integer or half-integer non-negative numbers")
        # Condition 8
        if not all(isinstance(x, (int, float, np.integer, np.floating)) and x >= 0 and x % 1 == 0 for x in
                   [j1 + m1, j2 + m2, j + m, j1 + j2 + j]):
            raise ValueError("Invalid input parameters: j1+m1,j2+m2,j+m,j1+j2+j must be integer non-negative numbers")
    def cg(self):
        if self.m1 + self.m2 != self.m:
            return 0.0  # delta function fails
        prefactor = cmath.sqrt((2 * self.j + 1) * fact(self.j + self.j1 - self.j2) * fact(self.j-self.j1 + self.j2) \
                               * fact(self.j1 + self.j2 - self.j) / fact(self.j + self.j1 + self.j2 + 1))
        coefficient = cmath.sqrt(fact(self.j + self.m) * fact(self.j - self.m) / (fact(self.j1 + self.m1) \
                               * fact(self.j1 - self.m1) * fact(self.j2 + self.m2) * fact(self.j2 - self.m2)))
        sum = 0.0
        smin= max(0, int(self.m1-self.j1),int(self.j2-self.j1+self.m))
        smax= min(int(self.j2+self.j+self.m1),int(self.j-self.j1+self.j2),\
                   int(self.j+self.m))

        for s in range(smin,smax+1):
            den = fact(s) * fact(self.j - self.j1 + self.j2 - s) * fact(self.j + self.m - s) \
                  * fact(self.j1 - self.j2 - self.m + s)
            num = ((-1) ** (self.j2 + self.m2 + s))* fact(self.j2 + self.j + self.m1 - s) * fact(self.j1 - self.m1 + s)
            sum += num / den
        cb = prefactor * coefficient * sum
        return cb

time: 4.8 ms (started: 2023-03-15 04:28:29 +00:00)


In [14]:
j1,m1,j2,m2,j,m=1,1,3/2,1/2,5/2,3/2
CG_calc = Clebsch_Gordan(j1,j2,j,m1,m2,m)
cg_calc = CG_calc.cg()
print(cg_calc)

(0.7745966692414833+0j)
time: 3.04 ms (started: 2023-03-15 04:28:42 +00:00)


In [11]:
from sympy.physics.quantum.cg import CG
from sympy import *
cg = CG(j1,m1,j2,m2,j,m)
cg = cg.doit()
print (N(cg))

0.774596669241483
time: 11.7 ms (started: 2023-03-15 04:23:58 +00:00)


**Comment:** our function to calculate Clebsch Gordan Coefficient run faster

### **Compute Coupling Coefficients $H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}$**



$$
C^{jm}_{{j_1}{m_1}{j_2}{m_2}} C^{jm'}_{{j_1}{m'_1}{j_2}{m'_2}} \equiv H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}   
$$

Compute function generate $m_1,m_2,m,m_1'm_2',m'$ that satisfies conditions check to calculate coupling coefficient $H_{j_1m_1m_1',j_2m_2m_2'}^{jmm'}$ via Clebsch Gordan coefficient

In [36]:
from itertools import product
import numpy as np
def generate_m_val(j1, j2, j):
    """
    This function generates (m1, m2, m, m1p, m2p, mp) from input set (j1,j2,j)
    and only keep set that satisfy the condition 
    """
    # Generate m values
    m  = np.linspace(-j, j, int(2 * j + 1))
    mp = m.copy()
    m1 = np.linspace(-j1, j1, int(2* j1 +1))
    m1p = m1.copy()
    m2 = np.linspace(-j2,j2, int(2 * j2 +1))
    m2p = m2.copy()
    s = product(m1, m2, m, m1p, m2p, mp)
    keep_list = []
    for i in s:
        m1, m2, m, m1p, m2p, mp = i[0], i[1], i[2], i[3], i[4], i[5]
        # Check input parameter conditions
        # Condition 1 & 2 & 5
        if not (abs(j1 - j2) <= j <= j1 + j2 and m1 + m2 == m and j1 + j2 - j % 1 != 0.5):
          pass
        if not (abs(j1 - j2) <= j <= j1 + j2 and m1p + m2p == mp and 
                j1 + j2 - j % 1 != 0.5):
          pass
        # Condition 3 & 6
        if not all(abs(x) <= y and (x % 0.5 == 0 or x % 1 == 0) for x, y in 
                     zip([m1, m2, m], [j1, j2, j])):
          pass
        if not all(abs(x) <= y and (x % 0.5 == 0 or x % 1 == 0) for x, y in 
                     zip([m1p, m2p, mp], [j1, j2, j])):
          pass
        # Condition 4
        J = (j1+j2+j)
        if J < (int(j1+j2+j)) and J <0:
          raise ValueError("Invalid input parameters: j1, j2, j \
                              must not exceed a positive integer J")
        # Condition 7
        if not all(isinstance(x, (int, float, np.integer, np.floating)) and x >= 0
                     and (x % 0.5 == 0 or x % 1 == 0) for x in [j1, j2, j]):
            raise ValueError("Invalid input parameters: j1, j2, j must be integer \
                             or half-integer non-negative numbers")
        # Condition 8
        if not all(isinstance(x, (int, float, np.integer, np.floating)) and x >= 0
                     and x % 1 == 0 for x in [j1 + m1, j2 + m2, j + m, j1 + j2 + j]):
          pass
        if not all(isinstance(x, (int, float, np.integer, np.floating)) and x >= 0 
                     and x % 1 == 0 for x in [j1 + m1p, j2 + m2p, j + mp, j1 + j2 + j]):
          pass
        else:
          keep_list.append(i)
    return keep_list


time: 6.26 ms (started: 2023-03-15 05:55:29 +00:00)


In [44]:
import pandas as pd
value_set= generate_m_val(1,3/2,5/2)
print (value_set)
print (len(value_set))
list_columns = ['m1', 'm1p', 'm2', 'm2p', 'm', 'mp' ]
df =pd.DataFrame(value_set, columns=list_columns)
print(df) 

[(-1.0, -1.5, -2.5, -1.0, -1.5, -2.5), (-1.0, -1.5, -2.5, -1.0, -1.5, -1.5), (-1.0, -1.5, -2.5, -1.0, -1.5, -0.5), (-1.0, -1.5, -2.5, -1.0, -1.5, 0.5), (-1.0, -1.5, -2.5, -1.0, -1.5, 1.5), (-1.0, -1.5, -2.5, -1.0, -1.5, 2.5), (-1.0, -1.5, -2.5, -1.0, -0.5, -2.5), (-1.0, -1.5, -2.5, -1.0, -0.5, -1.5), (-1.0, -1.5, -2.5, -1.0, -0.5, -0.5), (-1.0, -1.5, -2.5, -1.0, -0.5, 0.5), (-1.0, -1.5, -2.5, -1.0, -0.5, 1.5), (-1.0, -1.5, -2.5, -1.0, -0.5, 2.5), (-1.0, -1.5, -2.5, -1.0, 0.5, -2.5), (-1.0, -1.5, -2.5, -1.0, 0.5, -1.5), (-1.0, -1.5, -2.5, -1.0, 0.5, -0.5), (-1.0, -1.5, -2.5, -1.0, 0.5, 0.5), (-1.0, -1.5, -2.5, -1.0, 0.5, 1.5), (-1.0, -1.5, -2.5, -1.0, 0.5, 2.5), (-1.0, -1.5, -2.5, -1.0, 1.5, -2.5), (-1.0, -1.5, -2.5, -1.0, 1.5, -1.5), (-1.0, -1.5, -2.5, -1.0, 1.5, -0.5), (-1.0, -1.5, -2.5, -1.0, 1.5, 0.5), (-1.0, -1.5, -2.5, -1.0, 1.5, 1.5), (-1.0, -1.5, -2.5, -1.0, 1.5, 2.5), (-1.0, -1.5, -2.5, 0.0, -1.5, -2.5), (-1.0, -1.5, -2.5, 0.0, -1.5, -1.5), (-1.0, -1.5, -2.5, 0.0, -1.5, -0.5), 

In [15]:
def H(j1,j2,j,m1,m2,m,m1p,m2p,mp):
  '''
  This function calculate coupling coefficient H via computing
  the Clebsch-Gordan coefficient for cg(j1,m1,j2,m2,j,m)
  and cg(j1,m1p,j2,m2p,j,mp)
  Parameters:
        j1: angular momentum 1
        j2: angular momentum 2
        j: total angular momentum (j1+j2)
        m1: eigenvalue of angular momentum j1
        m2: eigenvalue of angular momentum j2
        m: eigenvalue of angular momentum j
        m1p: eigenvalue of j1 along rotated axis
        m2p: eigenvalue of j2 along rotated axis
        mp: eigenvalue of j along rotated axis
  Returns: Coupling coefficient H(j1,j2,j,m1,m2,m.m1p,m2p,mp)
  ======================Reference=========================
    [1] Thompson, Swiler, Trott, Foiles, Tucker,
        Spectral neighbor analysis method for automated generation of quantum-accurate interatomic potentials (2015)
    [5] Chapter 8  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        Quantum Theory of Angular Momentum (1988)
  '''
  CG = Clebsch_Gordan(j1,j2,j,m1,m2,m)
  cg = CG_calc.cg()
  CGp = Clebsch_Gordan(j1,j2,j,m1p,m2p,mp)
  cg_p = CGp.cg()
  H = (cg)*(cg_p)
  return H

time: 1.25 ms (started: 2023-03-15 04:29:38 +00:00)


Let try to compare how out function calculate $H_{j_1m_1m_1',j_2m_2m_2'}^{jmm'}$

In [16]:
import numpy as np
j = 3
j1 = 1
j2 = 2
m = np.linspace(-j, j, int(2 * j + 1))
mp = np.linspace(-j, j, int(2 * j + 1))
m1 = np.linspace(-j1, j1, int(2 * j1 + 1))
m1p = np.linspace(-j1, j1, int(2 * j1 + 1))
m2 = np.linspace(-j2, j2, int(2 * j2 + 1))
m2p = np.linspace(-j2, j2, int(2 * j2 + 1))
print(m1, m2, m, m1p, m2p, mp)

[-1.  0.  1.] [-2. -1.  0.  1.  2.] [-3. -2. -1.  0.  1.  2.  3.] [-1.  0.  1.] [-2. -1.  0.  1.  2.] [-3. -2. -1.  0.  1.  2.  3.]
time: 8.45 ms (started: 2023-03-15 04:34:20 +00:00)
